In [1]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

Using TensorFlow backend.


In [4]:
#stop word removal
from nltk.corpus import stopwords
stop={'a', 'of', 'on', 'the','a','an','the','and','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','only','own','same','so','than','too','very','will','just','should','now' } #add related stop word here
print(stop)

{'why', 'few', 'now', 'same', 'up', 'out', 'too', 'each', 'the', 'than', 'over', 'for', 'once', 'an', 'until', 'and', 'some', 'so', 'against', 'more', 'down', 'most', 'any', 'below', 'when', 'then', 'as', 'about', 'by', 'after', 'should', 'between', 'because', 'to', 'again', 'during', 'there', 'from', 'off', 'above', 'a', 'into', 'only', 'further', 'will', 'just', 'how', 'in', 'where', 'before', 'other', 'under', 'while', 'at', 'if', 'here', 'very', 'both', 'with', 'or', 'own', 'on', 'through', 'all', 'of'}


In [5]:
df_csv3 = pd.read_csv('2019 olid/olid-training-v1.0.tsv',sep='\t')
df_csv4 = pd.read_csv('task_b_distant_TIN.csv',sep=',')
df_csv5 = pd.read_csv('task_b_distant_UNT.csv',sep=',')
df_test = pd.read_csv('English/test_b_tweets.tsv',sep='\t')

df_csv4.head()

id                                               text  \
0  1159533712079503361  @USER Trump is a fucking idiot his dementia is...   
1  1159533718345830400  @USER Can't wait to see the shit show his deat...   
2  1159533739871002625  @USER @USER @USER This guys is dumb check his ...   
3  1159534097754132480  @USER Exactly, a pervert.  So disgusting and h...   
4  1159469570912137216                               Invest in your bitch   

    average       std  label  
0  0.230133  0.219593      1  
1  0.229661  0.228530      1  
2  0.169093  0.180201      1  
3  0.160362  0.188917      1  
4  0.208311  0.212202      1

In [11]:
#marge all data from files that has been read
q_a=[]
label=[]
av=[]
st=[]
for i in range(len(df_csv3)):
    label.append(df_csv3['subtask_b'][i])
    q_a.append(df_csv3['tweet'][i])
    
for i in range(len(df_csv4)):
    label.append(df_csv4['label'][i])
    q_a.append(df_csv4['text'][i])
    av.append(df_csv4['average'][i])
    st.append(df_csv4['std'][i])


for i in range(len(df_csv5)):
    label.append(df_csv5['label'][i])
    q_a.append(df_csv5['text'][i])

    
#marge all data from files that has been read
twee=[]
ids=[]
for i in range(len(df_test)):
    ids.append(df_test['id'][i])
    twee.append(df_test['tweet'][i])

In [12]:
print(len(q_a))
print(max(av))
print(min(av))
# print(max(st))
# print(min(st))

21908
0.2499943147613396
0.12208811343194945


In [42]:
# remove empty space from the begining of a sentance
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=" ".join(sen.split())
    QA.append(sen)
print(QA)
q_a=QA

['She should ask a few native Americans what their take on this is.', 'Go home you’re drunk!!! MAGA Trump2020 👊🇺🇸👊 URL', 'Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL Amazon MAGA KAG CHINA TCOT', 'Someone should\'veTaken" this piece of shit to a volcano. 😂"', 'Obama wanted liberals &amp; illegals to move into red states', 'Liberals are all Kookoo !!!', 'Oh noes! Tough shit.', 'was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism', 'Buy more icecream!!!', 'Canada doesn’t need another CUCK! We already have enough LooneyLeft Liberals f**king up our great country! Qproofs TrudeauMustGo', 'It’s not my fault you support gun control', 'What’s the difference between Kavanaugh and One of these men admitted to groping a 15 year old girl years ago. The other is going to be con

In [10]:
# #this code will stem text
stem_both_q_a=[]
for i in q_a:
    ps = PorterStemmer() #stemming word removing
    words = str(i).split(' ') 
    s=''
    for w in words: 
        stem=ps.stem(w)
        s=s+' '+stem
    s=s.strip()

    stem_both_q_a.append(s)
q_a=stem_both_q_a

# print(q_a)

In [71]:
# remove empty space from the begining of a sentance
TWEETS=[]
for i in twee:
    sen1=i.replace('@USER','')
    sen1=sen1.replace('@user','')
    sen1=sen1.replace("#", ' ')
    sen1=sen1.replace('@USER','')
    sen1=sen1.replace("NEWLINE_TOKEN", " ")
    sen1=sen1.replace("@", "")
    sen1=sen1.replace("TAB_TOKEN", " ")
    sen1=sen1.replace(" w ", ' with ')
    sen1=sen1.replace(" mf ", ' motherfucker ')
    sen1=sen1.replace(" IDGAF ", ' i dont give a fuck ')
    sen1=sen1.replace(" IDRC ", ' I Dont Really Care ')
    sen1=sen1.replace(" tf ", ' the fuck ')
    sen1=sen1.replace(" wtf ", ' what the fuck ')
    sen1=sen1.replace(" yanno ", ' you know ')
    sen1=sen1.replace(" igt ", ' i know right ')
    sen1=sen1.replace(" r ", ' are ')
    sen1=sen1.replace(" tbh ", ' to be honest ')
    sen1=sen1.replace(" ik ", ' i know ')
    sen1=sen1.replace(" af ", ' as fuck ')
    sen1=sen1.replace(" des ", ' this ')
    sen1=sen1.replace(" bout ", ' about ')
    sen1=sen1.replace(" em ", ' them ')
    sen1=sen1.replace(" stg ", ' swear to god ')
    sen1=sen1.replace(" bj ", ' blow job ')
    sen1=sen1.replace(" ig ", ' i guess ')
    sen1=sen1.replace(" fagg ", ' faggot ')
    sen1=sen1.replace(" fag ", ' faggot ')
    sen1=sen1.replace(" tfw ", ' That Feel When ')
    sen1=sen1.replace(" wa ", ' was ')
    sen1=sen1.replace(" n ", ' and ')
    sen1=sen1.replace(" y0 ", ' you ')
    sen1=sen1.replace(" dat ", ' that ')
    sen1=sen1.replace(" YO ", ' you ')
    sen1=" ".join(sen1.split())
    TWEETS.append(sen1)
twee=TWEETS
len(twee)

1422

In [43]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=label

In [44]:
dataframe = comments_attack

In [45]:
# Text preprocessing
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace('@USER',''))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" w ", ' with '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" mf ", ' motherfucker '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDGAF ", ' i dont give a fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDRC ", ' I Dont Really Care '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tf ", ' the fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wtf ", ' what the fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" yanno ", ' you know '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" igt ", ' i know right '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" r ", ' are '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tbh ", ' to be honest '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ik ", ' i know '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" af ", ' as fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" des ", ' this '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bout ", ' about '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" em ", ' them '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" stg ", ' swear to god '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bj ", ' blow job '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ig ", ' i guess '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fagg ", ' faggot '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fag ", ' faggot '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tfw ", ' That Feel When '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wa ", ' was '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" n ", ' and '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" y0 ", ' you '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" dat ", ' that '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" YO ", ' you '))

In [46]:
# this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


In [46]:
print(dataframe['label'])

0        UNT
1        TIN
2        NaN
3        UNT
4        NaN
        ... 
13235    TIN
13236    NaN
13237    TIN
13238    UNT
13239    NaN
Name: label, Length: 13240, dtype: object


In [47]:
comments_attack['label'][990]==0

False

In [48]:
print(dataframe['comment'])

0        she should ask a few native americans what the...
1                   go home youre drunk maga trump2020 url
2        amazon is investigating chinese employees who ...
3        someone shouldvetaken this piece of shit to a ...
4        obama wanted liberals amp illegals to move int...
                               ...                        
13235    sometimes i get strong vibes from people and t...
13236    benidorm creamfields maga not too shabby of a ...
13237      and why report this garbage we dont give a crap
13238                                                pussy
13239    spanishrevenge vs justice humanrights and free...
Name: comment, Length: 13240, dtype: object


In [29]:
# for x in dataframe['comment']:
#     print(x)

In [49]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.2, random_state=42)

In [50]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  10592
test comments length:  2648


In [51]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)


In [64]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [52]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [17]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NameError: name 'xtrain_count' is not defined

In [53]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Input contains NaN

In [89]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [28]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [29]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

NameError: name 'train_seq_x' is not defined

In [85]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [86]:
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
6934/6934 [==============================] - 9s 1ms/step - loss: 0.5988 - accuracy: 0.7437
Epoch 2/3
6934/6934 [==============================] - 8s 1ms/step - loss: 0.3421 - accuracy: 0.8750
Epoch 3/3
6934/6934 [==============================] - 7s 1ms/step - loss: 0.2126 - accuracy: 0.9282
LSTM, Word Embeddings accuracy:0.9509803652763367     f1 score: 0.9494653134649798


In [87]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [88]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# Naive Bayes on Word Level TF IDF Vectors

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# RF on Word Level TF IDF Vectors ngram
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print(accuracy)

# RF on Word Levelon Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)

print(accuracy)


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8252595155709342


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8304498269896193


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.828719723183391


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9290657439446367


In [121]:
clf = Pipeline([
    ('vect',tfidf_vect_ngram_chars),
    ('clf', linear_model.LogisticRegression()),
])

clf = clf.fit(train_x, train_y)

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [122]:
Labels=[]
IDE=[]
for i in range(len(twee)):
    IDE.append(ids[i])
    predictions = clf.predict([twee[i]])
    if predictions[0]==1:
        Labels.append('TIN')
    elif predictions[0]==0:
        Labels.append('UNT')
print(len(Labels))
print(len(IDE))

1422
1422


In [64]:
my_dict = { 'q_a' : IDE,'label' : Labels}
df = pd.DataFrame(my_dict)
df.to_csv('English_submission_task_b',index=False)